# Webcam analysis

In [1]:
import os
import pandas

from geopy.geocoders import Nominatim
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"

from util import get_canton_from_latlon
from get_data_json import get_cam_loc, get_cam_frame

In [2]:
cam_loc_json = get_cam_loc()
cam_info_json = get_cam_frame()

In [3]:
loc_col_example = list(cam_loc_json.values())[0]
loc_col_example

{'cam': {'longitude': 8.50748,
  'latitude': 47.310213,
  'altitude': 800,
  'angle': 180,
  'north': 30},
 'instances': {'515': {'url': 'https://laf.roundshot.com/',
   'title': 'Adliswil-Felsenegg Luftseilbahn LAF AG'}}}

## Webcam location info

In [4]:
cam_id = list(cam_loc_json.keys())[0]

longitude = list(cam_loc_json.values())[0]['cam']['longitude']
latitude = list(cam_loc_json.values())[0]['cam']['latitude']
altitude = list(cam_loc_json.values())[0]['cam']['altitude']
angle = list(cam_loc_json.values())[0]['cam']['angle']
north = list(cam_loc_json.values())[0]['cam']['north']

# loc_col_example = list(cam_loc_json.values())[0]
instance_id = list(loc_col_example['instances'].keys())[0]

cam_link = list(loc_col_example['instances'].values())[0]['url']

cam_name = list(loc_col_example['instances'].values())[0]['title']

cam_link_ = cam_link.split('/')
cam_abbr = cam_link_[2].split('.')[0]
if len(cam_link_) > 4:
    cam_abbr = cam_abbr + '_' + cam_link_[3]

print('cam_id:', cam_id)
print('longitude:', longitude)
print('latitude:', latitude)
print('altitude:', altitude)
print('angle:', angle)
print('north:', north)
print('longitude:', longitude)
print('instance_id:', instance_id)
print('cam_link:', cam_link)
print('cam_name:', cam_name)
print('cam_abbr:', cam_abbr)

cam_id: 467
longitude: 8.50748
latitude: 47.310213
altitude: 800
angle: 180
north: 30
longitude: 8.50748
instance_id: 515
cam_link: https://laf.roundshot.com/
cam_name: Adliswil-Felsenegg Luftseilbahn LAF AG
cam_abbr: laf


In [5]:
canton1, _ = get_canton_from_latlon(46.07844, 7.77628) # Zermatt
canton2, _ = get_canton_from_latlon(46.507854, 6.498673) # Morges - Port
canton3, _ = get_canton_from_latlon(47.36931, 8.541663) # Zurich Stadthaus
canton1, canton2, canton3

('Valais', 'Vaud', 'Zürich')

## Webcam info

In [6]:
for k in cam_info_json['categories'].keys():
    print(k, len(cam_info_json['categories'][k]))
print('All', len(cam_loc_json.values()))

Schools 5
Hospitals 11
Airports 10
Tourism associations 63
Golf clubs 1
Museums 1
Restaurants 5
Harbors 5
Villages 28
Ski resorts 131
Weather cams 4
Corporates 28
Construction sites 22
Cities 9
Hotels 21
All 345


In [7]:
info_col_names = cam_info_json['categories']['Cities'][0].keys()
info_col_names

dict_keys(['picture', 'link', 'angle', 'name', 'cam_name', 'customer_name'])

In [8]:
category = list(cam_info_json['categories'].keys())[0]
cam_info_example = cam_info_json['categories'][category][0]
thumbnail_url = cam_info_example["picture"]
cam_link = cam_info_example["link"]
angle = cam_info_example["angle"]
name = cam_info_example["name"] # cam_name - customer_name or equal to cam_name/customer_name
cam_name = cam_info_example["cam_name"]
customer_name = cam_info_example["customer_name"]
cam_link_ = cam_link.split('/')
cam_abbr = cam_link_[2].split('.')[0]
if len(cam_link_) > 4:
    cam_abbr = cam_abbr + '_' + cam_link_[3]

print('category:', category)
print('thumbnail_url:', thumbnail_url)
print('cam_link:', cam_link)
print('angle:', angle)
print('name:', name)
print('cam_name:', cam_name)
print('customer_name:', customer_name)
print('cam_abbr:', cam_abbr)

category: Schools
thumbnail_url: https://backend.roundshot.com/cams/514/thumbnail
cam_link: https://aigloncollege.roundshot.com/
angle: 360
name: Aiglon College
cam_name: Aiglon College
customer_name: Aiglon College
cam_abbr: aigloncollege


In [9]:
import requests

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"
}
response = requests.get(thumbnail_url, stream=True, headers=headers)

In [10]:
# print(dir(response))
print(response.status_code)
print(response.url)
# https://stackoverflow.com/a/35616488
webcam_stored_url = response.url.rsplit('/', 3)[0]
print('webcam_stored_url of', cam_abbr, webcam_stored_url)

200
https://storage2.roundshot.com/576c4aa94999f3.08977207/2021-12-26/16-50-00/2021-12-26-16-50-00_thumbnail.jpg
webcam_stored_url of aigloncollege https://storage2.roundshot.com/576c4aa94999f3.08977207


## Test

In [11]:
# https://github.com/geopy/geopy#geocoding
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"

geolocator = Nominatim(user_agent=user_agent)
latitude, longitude = 47.310213, 8.50748 # example
# Felseneggbahn, 8, Felseneggweg, Buchenegg, Stallikon, Bezirk Affoltern, Zürich, 8143, Schweiz/Suisse/Svizzera/Svizra
location = geolocator.reverse(
    query =(latitude, longitude),
    exactly_one = True,
    timeout=10,
)
print(location.address)
# print('address raw:', location.raw)

Felseneggbahn, 8, Felseneggweg, Buchenegg, Stallikon, Bezirk Affoltern, Zürich, 8143, Schweiz/Suisse/Svizzera/Svizra


In [12]:
location.address.replace(" ", "").split(",")

['Felseneggbahn',
 '8',
 'Felseneggweg',
 'Buchenegg',
 'Stallikon',
 'BezirkAffoltern',
 'Zürich',
 '8143',
 'Schweiz/Suisse/Svizzera/Svizra']